In [652]:
import random
import pickle
from tqdm import tqdm 
import statistics
import sys
import os
import configparser
import json
import pandas as pd
import numpy as np

from dataset import dataset
from model import FNN
import torch.nn as nn
import torch
from torch.utils.data import DataLoader

### stock price prediction schema
* train 2013 Q1 ~ 2022 Q1
* predict 2022 Q1 stock return

In [653]:
start_Year = '2013'
start_Quarter = '1'

end_Year = '2022'
end_Quarter = '1'

In [654]:
df = pd.read_csv('ratio.csv', dtype='str', keep_default_na=True)
df

,CO_ID,Name,Year,Quarter,Sector,Current Ratio,Long Term Debt / Captial,Debt / Equity,Gross Margin,Operating Margin,...,Days Sales In Receivables,ROE,ROTE,ROA,ROI,Book Value Per Share,Operating Cash Flow Per Share,Free Case Flow Per Share,Stock Return,Relative Return
0,1101.TW,台泥,2013,1,Basic Materials,1.171943815834016,0.29442130707435726,0.4172763577278045,0.12954573738700934,0.08404765073237189,...,143.64380956046077,0.013755817414366902,0.015838087454927118,0.007329053429107523,0.02512258672597047,39.371281850358216,0.615911338352,0.6731827159155662,-0.011464985321349055,0.0434235362020827
1,1101.TW,台泥,2013,2,Basic Materials,1.118428250520514,0.3139175390369567,0.45755074192731227,0.196092532925495,0.1600614771080646,...,114.15108863619432,0.027312566260955757,0.03175802191577107,0.013894325028438818,0.04587993732988118,37.70328710503744,1.6331705648151813,1.913097910638548,0.14621416232432163,0.10592110693198141
2,1101.TW,台泥,2013,3,Basic Materials,1.0471787876947094,0.2616990972544916,0.3544612992904588,0.2343151667300576,0.19632197189482428,...,100.0781717735296,0.03680989073858024,0.04243794402067692,0.019650002607972096,0.07560443949052767,39.32541675492763,1.3999235518546125,1.527473000406075,0.07638080274401479,0.046265643488684234
3,1101.TW,台泥,2013,4,Basic Materials,1.2470160938385983,0.2707074824861595,0.3711919099471934,0.21147473534667693,0.15808357758010247,...,95.4223535578622,0.025858148653466567,0.029616862763056435,0.014052220232804754,0.05211201407036397,41.606516363426785,2.3023916601589867,2.5973759809222523,-0.02944382478915979,-0.1022971957705434
4,1101.TW,台泥,2014,1,Basic Materials,1.2383782787048268,0.2455049419764561,0.32538972835630575,0.18206665185170046,0.13790210720997392,...,109.97321309629538,0.01763756744648356,0.02010746683852985,0.009767680425940554,0.03939816083527232,42.86228616572683,1.8676878314492005,2.1224516957602155,0.042792818836951164,-0.04818949460438377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3584,9945.TW,潤泰新,2021,1,Industrials,1.3732725181743248,0.26912342159292485,0.36822006552661973,0.19539411944969245,0.10960422420516913,...,83.34654558958727,0.06938477273778565,0.06956422714496523,0.041237684017517484,0.043694566957971616,65.88029086685259,0.4164402368027771,0.47861724614770207,0.24705883461711367,0.23709712540824154
3585,9945.TW,潤泰新,2021,2,Industrials,1.8069328172791335,0.2894056604893004,0.407272679217483,0.22063500006609274,0.1468273601241788,...,88.43072647869894,0.03570387734181559,0.03576324401294579,0.021911234424886523,0.022089643578290862,53.551786771559634,-0.40254772873830463,-0.3616792860792885,0.4911111195882161,0.3989504843931277
3586,9945.TW,潤泰新,2021,3,Industrials,1.6762313564665816,0.27969034439771956,0.3882918161965481,0.21301992250737792,0.14887966820659174,...,90.85896217608521,0.040216642039633946,0.04028535507930397,0.024437036262350564,0.025429212669564293,53.14985845582285,0.9563713011194905,0.9845291991288918,0.07796103003688017,0.06848397751011784
3587,9945.TW,潤泰新,2021,4,Industrials,2.003309519134619,0.2593057124467081,0.3500846662436983,0.20745287948705896,0.11460756637792076,...,66.98461826647386,0.016896930017664985,0.016923251367797845,0.010846204085113676,0.01144088132158306,58.43108050214127,2.0290821286581084,2.0704361341528004,0.0,0.14255910684940287


In [655]:
df.columns

Index(['CO_ID', 'Name', 'Year', 'Quarter', 'Sector', 'Current Ratio',
       'Long Term Debt / Captial', 'Debt / Equity', 'Gross Margin',
       'Operating Margin', 'Pre-Tax Profit Margin', 'Net Profit Margin',
       'Asset Turnover', 'Inventory Turnover', 'Receivable Turnover',
       'Days Sales In Receivables', 'ROE', 'ROTE', 'ROA ', 'ROI',
       'Book Value Per Share', 'Operating Cash Flow Per Share',
       'Free Case Flow Per Share', 'Stock Return', 'Relative Return'],
      dtype='object')

In [656]:
# ratio columns
ratio_columns = df.columns[5:-2]
df[df.columns[5:-2]]

,Current Ratio,Long Term Debt / Captial,Debt / Equity,Gross Margin,Operating Margin,Pre-Tax Profit Margin,Net Profit Margin,Asset Turnover,Inventory Turnover,Receivable Turnover,Days Sales In Receivables,ROE,ROTE,ROA,ROI,Book Value Per Share,Operating Cash Flow Per Share,Free Case Flow Per Share
0,1.171943815834016,0.29442130707435726,0.4172763577278045,0.12954573738700934,0.08404765073237189,0.09707159482603646,0.08292361709338959,0.08838318546637267,2.0683745313185162,2.541007517949242,143.64380956046077,0.013755817414366902,0.015838087454927118,0.007329053429107523,0.02512258672597047,39.371281850358216,0.615911338352,0.6731827159155662
1,1.118428250520514,0.3139175390369567,0.45755074192731227,0.196092532925495,0.1600614771080646,0.16075146221593833,0.132615298178763,0.10477166073034441,2.2874321010591103,3.197516592796366,114.15108863619432,0.027312566260955757,0.03175802191577107,0.013894325028438818,0.04587993732988118,37.70328710503744,1.6331705648151813,1.913097910638548
2,1.0471787876947094,0.2616990972544916,0.3544612992904588,0.2343151667300576,0.19632197189482428,0.21650964420409596,0.17752318667529327,0.11068978073221393,2.385870504011114,3.6471489589755026,100.0781717735296,0.03680989073858024,0.04243794402067692,0.019650002607972096,0.07560443949052767,39.32541675492763,1.3999235518546125,1.527473000406075
3,1.2470160938385983,0.2707074824861595,0.3711919099471934,0.21147473534667693,0.15808357758010247,0.15467631369509108,0.11961896871636811,0.1174748485428291,2.81979430397297,3.8250995326652824,95.4223535578622,0.025858148653466567,0.029616862763056435,0.014052220232804754,0.05211201407036397,41.606516363426785,2.3023916601589867,2.5973759809222523
4,1.2383782787048268,0.2455049419764561,0.32538972835630575,0.18206665185170046,0.13790210720997392,0.13290971064140158,0.10447714042854929,0.09349107743449925,2.0502751562258545,3.318990049698708,109.97321309629538,0.01763756744648356,0.02010746683852985,0.009767680425940554,0.03939816083527232,42.86228616572683,1.8676878314492005,2.1224516957602155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3584,1.3732725181743248,0.26912342159292485,0.36822006552661973,0.19539411944969245,0.10960422420516913,1.3408053751502405,1.313418686819025,0.03139721128636538,0.14361428184867386,4.379305673894666,83.34654558958727,0.06938477273778565,0.06956422714496523,0.041237684017517484,0.043694566957971616,65.88029086685259,0.4164402368027771,0.47861724614770207
3585,1.8069328172791335,0.2894056604893004,0.407272679217483,0.22063500006609274,0.1468273601241788,-0.4497400982075022,0.6339046577749914,0.034565504695603703,0.16529753584487633,4.127524612024087,88.43072647869894,0.03570387734181559,0.03576324401294579,0.021911234424886523,0.022089643578290862,53.551786771559634,-0.40254772873830463,-0.3616792860792885
3586,1.6762313564665816,0.27969034439771956,0.3882918161965481,0.21301992250737792,0.14887966820659174,0.07099250947165506,0.5318008242875873,0.04595148248423078,0.23472851579202894,4.01721515696633,90.85896217608521,0.040216642039633946,0.04028535507930397,0.024437036262350564,0.025429212669564293,53.14985845582285,0.9563713011194905,0.9845291991288918
3587,2.003309519134619,0.2593057124467081,0.3500846662436983,0.20745287948705896,0.11460756637792076,1.1658810713367378,0.287560101637087,0.0377180423270334,0.19841977239303857,5.449012167957436,66.98461826647386,0.016896930017664985,0.016923251367797845,0.010846204085113676,0.01144088132158306,58.43108050214127,2.0290821286581084,2.0704361341528004


In [657]:
mask = df.columns[5:]
df[mask] = df[mask].astype(np.float64)  # convert ratio columns to numeric type

df['Year'] = df['Year'].astype(np.int16)  # convert ratio columns to numeric type
df['Quarter'] = df['Quarter'].astype(np.int16)  # convert ratio columns to numeric type

## missing cells
* ratios

In [658]:
# missing cell: take average of values within all intervals if other values available, else -1

for col in ratio_columns:  # each ratio
    for co_id in df['CO_ID'].unique():    # each company
        # fill -1
        if df.loc[df["CO_ID"] == co_id, col].isnull().all():
            fill_val = -1
        # fill average
        else:
            fill_val = df.loc[df["CO_ID"] == co_id, col].mean(skipna=True)

        df.loc[df["CO_ID"] == co_id, col] = df.loc[df["CO_ID"] == co_id, col].fillna(fill_val)


In [659]:
for col in ratio_columns:  # each ratio
    print(f'{col}, empty count:', (df[col] == '').sum())

Current Ratio, empty count: 0
Long Term Debt / Captial, empty count: 0
Debt / Equity, empty count: 0
Gross Margin, empty count: 0
Operating Margin, empty count: 0
Pre-Tax Profit Margin, empty count: 0
Net Profit Margin, empty count: 0
Asset Turnover, empty count: 0
Inventory Turnover, empty count: 0
Receivable Turnover, empty count: 0
Days Sales In Receivables, empty count: 0
ROE, empty count: 0
ROTE, empty count: 0
ROA , empty count: 0
ROI, empty count: 0
Book Value Per Share, empty count: 0
Operating Cash Flow Per Share, empty count: 0
Free Case Flow Per Share, empty count: 0


In [660]:
for col in ratio_columns:  # each ratio
    print(f'{col}, NaN count', df[col].isna().sum())

Current Ratio, NaN count 0
Long Term Debt / Captial, NaN count 0
Debt / Equity, NaN count 0
Gross Margin, NaN count 0
Operating Margin, NaN count 0
Pre-Tax Profit Margin, NaN count 0
Net Profit Margin, NaN count 0
Asset Turnover, NaN count 0
Inventory Turnover, NaN count 0
Receivable Turnover, NaN count 0
Days Sales In Receivables, NaN count 0
ROE, NaN count 0
ROTE, NaN count 0
ROA , NaN count 0
ROI, NaN count 0
Book Value Per Share, NaN count 0
Operating Cash Flow Per Share, NaN count 0
Free Case Flow Per Share, NaN count 0


In [661]:
# co_id = "2882.TW"

# (df.loc[df["CO_ID"] == co_id, 'Current Ratio'] == '').mean()
# df.loc[df["CO_ID"] == co_id, 'Current Ratio'].isnull().all()

In [662]:
# df.loc[df["CO_ID"] == co_id, 'Current Ratio']

In [663]:
# avg = df.loc[df["CO_ID"] == co_id, 'Current Ratio'].mean(skipna=True)
# df.loc[df["CO_ID"] == co_id, 'Current Ratio'] = df.loc[df["CO_ID"] == co_id, 'Current Ratio'].fillna(avg)

## standardization
* ratios

In [664]:
# for col in ratio_columns:  # each ratio
#     for co_id in df['CO_ID'].unique():    # each company
#         avg = df.loc[df["CO_ID"] == co_id, col].mean()
#         std = df.loc[df["CO_ID"] == co_id, col].std()
        
#         df.loc[df["CO_ID"] == co_id, col] = (df.loc[df["CO_ID"] == co_id, col] - avg) / std

In [665]:
for col in ratio_columns:  # each ratio
    # print(df.loc[:, col].mean())
    print(df.loc[:, col].std())

1.3239412616665571
0.46023481935535915
0.6826813927007386
0.45819946448390864
0.41020627881496463
0.5582686341265952
0.5417914833844678
0.47684154183330013
1515.7172581967434
26.70145352530304
127.68317605425172
0.04437887970219911
0.3456280056726231
0.02274355160715798
1.42697259683821
74.15764495017865
6.74198238212704
7.875095437276074


In [666]:
for col in ratio_columns:  # each ratio
    avg = df.loc[:, col].mean()
    std = df.loc[:, col].std()
        
    df.loc[:, col] = (df.loc[:, col] - avg) / std

## One hot encoding
* Sector

In [667]:
df_sector = pd.get_dummies(df['Sector'], prefix='sec')

In [668]:
df_X = pd.concat([df[ratio_columns], df_sector], axis=1)
df_X

,Current Ratio,Long Term Debt / Captial,Debt / Equity,Gross Margin,Operating Margin,Pre-Tax Profit Margin,Net Profit Margin,Asset Turnover,Inventory Turnover,Receivable Turnover,...,Free Case Flow Per Share,sec_Basic Materials,sec_Communication Services,sec_Consumer Cyclical,sec_Consumer Defensive,sec_Energy,sec_Financial Services,sec_Industrials,sec_Real Estate,sec_Technology
0,-0.122729,0.614911,0.416900,0.226833,0.385745,0.253641,0.241398,0.067260,-0.038080,-0.083393,...,-0.287378,1,0,0,0,0,0,0,0,0
1,-0.163150,0.657272,0.475894,0.372069,0.571051,0.367707,0.333116,0.101629,-0.037936,-0.058806,...,-0.129931,1,0,0,0,0,0,0,0,0
2,-0.216966,0.543812,0.324887,0.455488,0.659447,0.467584,0.416004,0.114040,-0.037871,-0.041967,...,-0.178898,1,0,0,0,0,0,0,0,0
3,-0.066025,0.563385,0.349395,0.405640,0.566230,0.356825,0.309128,0.128269,-0.037584,-0.035302,...,-0.043039,1,0,0,0,0,0,0,0,0
4,-0.072549,0.508625,0.282303,0.341458,0.517031,0.317836,0.281180,0.077972,-0.038092,-0.054257,...,-0.103346,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3584,0.029339,0.559943,0.345041,0.370545,0.448047,2.481482,2.512558,-0.052247,-0.039350,-0.014547,...,-0.312085,0,0,0,0,0,0,1,0,0
3585,0.356892,0.604013,0.402246,0.425632,0.538789,-0.725837,1.258360,-0.045602,-0.039336,-0.023976,...,-0.418788,0,0,0,0,0,0,1,0,0
3586,0.258170,0.582903,0.374443,0.409012,0.543792,0.206926,1.069904,-0.021725,-0.039290,-0.028107,...,-0.247843,0,0,0,0,0,0,1,0,0
3587,0.505219,0.538611,0.318476,0.396862,0.460244,2.168148,0.619102,-0.038991,-0.039314,0.025515,...,-0.109951,0,0,0,0,0,0,1,0,0


In [669]:
# for co_id in df['CO_ID'].unique():
#     print(df.loc[(df["CO_ID"] == co_id) & (df["Year"] == Year) & (df["Quarter"] == Quarter), 'Sector'].values[0])

## X: sector & ratios

In [670]:
co_id_list = df['CO_ID'].unique()
N = len(co_id_list) # num of company
M = len(df.loc[(df["CO_ID"] == co_id_list[0])]) # num of entries per company
K = len(df_X.columns)
X = np.zeros((N, M, K), dtype=float)

In [671]:
X.shape

(97, 37, 27)

In [672]:
for i, co_id in enumerate(co_id_list):
    Y = int(start_Year)
    Q = int(start_Quarter)
    for j in range(M):
        X[i, j] = df_X.loc[(df["CO_ID"] == co_id) & (df["Year"] == Y) & (df["Quarter"] == Q)].values[0]

        Y += (Q == 4)
        Q = (Q%4) + 1


In [284]:
# Y = int(end_Year)
# Q = int(end_Quarter)
# for i in range(37):
#     print(Y, Q)
#     Y -= (Q == 1)
#     Q = ((Q-2)%4) + 1

# Y = int(start_Year)
# Q = int(start_Quarter)
# for i in range(37):
#     print(Y, Q)
#     Y += (Q == 4)
#     Q = (Q%4) + 1

In [285]:
# one row of training data

df_X.loc[(df["CO_ID"] == co_id) & (df["Year"] == 2022) & (df["Quarter"] == 1)].values[0]

array([ 0.72148291, -0.36316038, -0.42294866,  0.40901974,  1.12320543,
       -0.13683413, -0.17156779, -0.12200191,  0.96050709,  0.78354615,
       -0.78976596, -0.25129467, -0.25244405, -0.16307636, -0.21748032,
        0.19549874,  0.28376096,  0.25382731,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ])

## Y: stock return

In [428]:
co_id_list = df['CO_ID'].unique()
N = len(co_id_list) # num of company
M = len(df.loc[(df["CO_ID"] == co_id_list[0])]) # num of entries per company
Y = np.zeros((N, M), dtype=float)

In [429]:
for i, co_id in enumerate(co_id_list):
    Y[i] = df.loc[(df["CO_ID"] == co_id), 'Stock Return'].values

## split train & val data (generate index)
* step 1:
* step 2:

In [430]:
# train & val: 0~N-5
# train: 4/5
# val: 1/5
# test: last 4 quarters
co_id_dummy = df['CO_ID'][0]
N = len(df.loc[(df["CO_ID"] == co_id_dummy)]) # num of entries per company

N_step1 = N - 5
N_step2 = N - 1

In [431]:
val_idx = np.random.choice(range(N_step1*2//3, N_step1), size=N_step1//5, replace=False).tolist()

train_idx_step1 = list(set(range(N_step1)) - set(val_idx))
train_idx_step2 = train_idx_step1 + list(range(N_step1, N_step2))


In [433]:
print(sorted(val_idx))
print(train_idx_step1)
print(train_idx_step2)

[21, 22, 23, 24, 29, 31]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 26, 27, 28, 30]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 26, 27, 28, 30, 32, 33, 34, 35]


## X_test_step1

In [565]:
N, M, K = X.shape[0], 4, X.shape[2]
end = X.shape[1]-1
l = 4
# X_test = np.zeros((N * M, K), dtype=float)
# Y_test_rel = np.zeros((N * M), dtype=float)
# Y_test_abs = np.zeros((N * M), dtype=float)


In [578]:
X_test = X[:, end-l:end, :].reshape((N * M, K))
Y_test = Y[:, end-l:end].reshape((N * M))

In [586]:
print(X_test.shape)
print(Y_test.shape)

(388, 27)
(388,)


In [572]:
X_test = X[:, -5:-1, :].reshape((4*N, K))
print(X_test.shape)
X_test[-4:]

(388, 27)


array([[ 0.02933908,  0.55994329,  0.34504137,  0.37054454,  0.44804678,
         2.48148176,  2.51255826, -0.05224682, -0.03935009, -0.01454673,
        -0.65763091,  0.80257774,  0.07580288,  1.10828529, -0.08466377,
         0.2892392 , -0.24845241, -0.31208458,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ],
       [ 0.3568916 ,  0.60401261,  0.40224611,  0.42563165,  0.53878926,
        -0.72583701,  1.25835983, -0.04560249, -0.03933579, -0.02397622,
        -0.61781218,  0.043638  , -0.02199297,  0.25853017, -0.09980416,
         0.12299196, -0.36992825, -0.41878761,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ],
       [ 0.25817011,  0.58290314,  0.37444271,  0.40901208,  0.54379237,
         0.20692649,  1.06990387, -0.02172458, -0.03928998, -0.02810743,
        -0.59879452,  0.14532521, -0.00890922,  0.3695

In [673]:
X[-1, -5:-1, :]


array([[ 0.02933908,  0.55994329,  0.34504137,  0.37054454,  0.44804678,
         2.48148176,  2.51255826, -0.05224682, -0.03935009, -0.01454673,
        -0.65763091,  0.80257774,  0.07580288,  1.10828529, -0.08466377,
         0.2892392 , -0.24845241, -0.31208458,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ],
       [ 0.3568916 ,  0.60401261,  0.40224611,  0.42563165,  0.53878926,
        -0.72583701,  1.25835983, -0.04560249, -0.03933579, -0.02397622,
        -0.61781218,  0.043638  , -0.02199297,  0.25853017, -0.09980416,
         0.12299196, -0.36992825, -0.41878761,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ],
       [ 0.25817011,  0.58290314,  0.37444271,  0.40901208,  0.54379237,
         0.20692649,  1.06990387, -0.02172458, -0.03928998, -0.02810743,
        -0.59879452,  0.14532521, -0.00890922,  0.3695

In [681]:
X[:, -1, :].reshape(X.shape[0], X.shape[2]).shape

(97, 27)

In [216]:
tmp = np.array([[[1,2,3], [4,5,6]], [[7,8,9], [10, 11, 12]], [[13,14,15], [16,17,18]]])

## test snippets

In [607]:
np.argmax([19, -3, 2])

0